# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key
from pprint import pprint

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.48,95,100,16.48,TF,1726519117
1,1,bredasdorp,-34.5322,20.0403,8.62,86,0,0.82,ZA,1726519118
2,2,salekhard,66.5300,66.6019,13.97,90,96,1.52,RU,1726519120
3,3,shanga,11.2106,4.5788,25.74,88,100,2.52,NG,1726519121
4,4,jamestown,42.0970,-79.2353,27.23,40,0,4.47,US,1726518921


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot= city_data_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_range = (15, 40)
ideal_humidity = 70

# Drop any rows with null values
ideal_weather_df = city_data_df[
(city_data_df['Max Temp'].between(ideal_temp_range[0], ideal_temp_range[1])) &
(city_data_df['Humidity'] == ideal_humidity)
]


# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,remire-montjoly,4.9167,-52.2667,30.02,70,0,5.14,GF,1726519260
220,220,lazaro cardenas,17.9583,-102.2000,30.58,70,53,3.26,MX,1726519389
235,235,acapulco de juarez,16.8634,-99.8901,31.90,70,100,3.09,MX,1726519407
240,240,usogorsk,63.4106,48.6872,15.02,70,8,2.96,RU,1726519412
286,286,waingapu,-9.6567,120.2641,23.55,70,3,1.87,ID,1726519467
325,325,yanbu,24.0891,38.0637,30.02,70,0,3.09,SA,1726519514
362,362,road town,18.4167,-64.6167,31.02,70,75,4.12,VG,1726519559
407,407,taulaga,-11.0553,-171.0883,27.99,70,54,5.12,AS,1726519617
509,509,rio pardo,-29.9897,-52.3781,18.84,70,8,1.53,BR,1726519742


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,95,
1,bredasdorp,ZA,-34.5322,20.0403,86,
2,salekhard,RU,66.5300,66.6019,90,
3,shanga,NG,11.2106,4.5788,88,
4,jamestown,US,42.0970,-79.2353,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
latitude = -43.0167
longitude = 147.3167
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    data = {}
    
    if name_address.status_code == 200:

        data = name_address.json()
        print(data)
    else:
        print(f"Error: {name_address.status_code}")
   
    # Convert the API response to JSON format
    name_address = data

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
   
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    #print(f"Request URL: {base_url}?{requests.compat.urlencode(params)}")
# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Keravel', 'country': 'France', 'country_code': 'fr', 'state': 'French Southern and Antarctic Lands', 'city': 'Port-aux-Français', 'municipality': 'Kerguelen Archipelago', 'street': 'Chemin de Keravel', 'lon': 70.21991619089022, 'lat': -49.34887615, 'formatted': 'Chemin de Keravel, Port-aux-Français, France', 'address_line1': 'Chemin de Keravel', 'address_line2': 'Port-aux-Français, France', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': [], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database License', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Keravel', 'osm_id': 187212461, 'tourism': 'hotel', 'building': 'yes', 'osm_type': 'w', 'addr:city': 'Port-aux-Français', 'addr:street': 'Chemin de Keravel', 'addr:district': 'Îles Kerguelen', 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,95,Keravel
1,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
2,salekhard,RU,66.5300,66.6019,90,No hotel found
3,shanga,NG,11.2106,4.5788,88,No hotel found
4,jamestown,US,42.0970,-79.2353,40,DoubleTree Jamestown
...,...,...,...,...,...,...
533,bryne,NO,58.7354,5.6477,88,Jæren hotell
534,champerico,GT,14.3000,-91.9167,73,Hotel y Restaurante El Submarino
535,nyurba,RU,63.2842,118.3319,91,Марина
536,funadhoo,MV,6.1500,73.2667,79,JW Marriott Maldives Resort & Spa


In [7]:
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,95,Keravel
1,bredasdorp,ZA,-34.5322,20.0403,86,Victoria Hotel
2,salekhard,RU,66.5300,66.6019,90,No hotel found
3,shanga,NG,11.2106,4.5788,88,No hotel found
4,jamestown,US,42.0970,-79.2353,40,DoubleTree Jamestown


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot= hotel_df.hvplot.points(
    'Lng',
    'Lat',
    size='Humidity',
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)